# Airbnb NYC Real Estate Analysis - Exploratory Data Analysis

In [ ]:
pip install pyspark findspark

# 1️⃣ Environment Setup (Spark + Hadoop for Windows)

In [1]:
import os
import findspark

# Set Hadoop path (make sure it's correct)
os.environ['HADOOP_HOME'] = r'C:\Program Files\hadoop\hadoop-3.2.2'
os.environ['PATH'] += os.pathsep + r'C:\Program Files\hadoop\hadoop-3.2.2\bin'

# Initialize findspark
findspark.init()

# Start Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AirbnbAnalysis") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

print("✅ Spark Session Started Successfully")


✅ Spark Session Started Successfully


# 2️⃣ Load Processed Parquet Data

In [2]:
# Path to processed parquet file (same location where you saved parquet from previous notebook)
parquet_path = r"D:\Darshana\Projects\Real_Estate_Market_Data\data\processed\airbnb.parquet"

df = spark.read.parquet(parquet_path)

# Show schema
df.printSchema()

# Preview sample rows
df.show(5)


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews_ltm: integer (nullable = true)
 |-- license: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- beds: integer (nullable = true)
 |-- baths: string (nullable = true)
 |-- price_per_night: double (nullable = true)


# 3️⃣ Register Temp Table for Spark SQL Queries

In [3]:
# Create SparkSQL view for querying
df.createOrReplaceTempView("airbnb")

# 4️⃣ Exploratory Data Analysis Using SparkSQL

In [4]:
# Total Listings
spark.sql("SELECT COUNT(*) as total_listings FROM airbnb").show()

+--------------+
|total_listings|
+--------------+
|         20758|
+--------------+



In [5]:
# Average Price Per Night
spark.sql("SELECT AVG(price_per_night) as avg_price FROM airbnb").show()

+------------------+
|         avg_price|
+------------------+
|187.77661624433952|
+------------------+



In [6]:
# Listings and Price by Room Type
spark.sql("""
SELECT room_type, COUNT(*) as total_listings, ROUND(AVG(price_per_night), 2) as avg_price
FROM airbnb
GROUP BY room_type
ORDER BY total_listings DESC
""").show()

+---------------+--------------+---------+
|      room_type|total_listings|avg_price|
+---------------+--------------+---------+
|Entire home/apt|         11549|   223.16|
|   Private room|          8804|   139.52|
|    Shared room|           293|   156.29|
|     Hotel room|           112|   415.05|
+---------------+--------------+---------+



In [7]:
# Top Neighborhoods by Average Price
spark.sql("""
SELECT neighbourhood, COUNT(*) as total_listings, ROUND(AVG(price_per_night), 2) as avg_price
FROM airbnb
GROUP BY neighbourhood
ORDER BY avg_price DESC
LIMIT 10
""").show()


+------------------+--------------+---------+
|     neighbourhood|total_listings|avg_price|
+------------------+--------------+---------+
|           Tribeca|            71|   455.41|
|          Longwood|            31|   424.23|
|      Civic Center|             8|   393.75|
|              SoHo|           136|   363.51|
|              NoHo|             9|   351.33|
|  Theater District|           140|   347.26|
|      West Village|           211|   341.79|
| Flatiron District|            39|   332.62|
|Financial District|           204|   314.87|
|             DUMBO|            17|   312.88|
+------------------+--------------+---------+



In [8]:
# Price Bucketing Distribution
spark.sql("""
SELECT 
  CASE 
    WHEN price_per_night < 50 THEN '<$50'
    WHEN price_per_night BETWEEN 50 AND 100 THEN '$50-100'
    WHEN price_per_night BETWEEN 100 AND 200 THEN '$100-200'
    ELSE '>$200' 
  END as price_bucket,
  COUNT(*) as total_listings
FROM airbnb
GROUP BY price_bucket
ORDER BY total_listings DESC
""").show()


+------------+--------------+
|price_bucket|total_listings|
+------------+--------------+
|    $100-200|          7994|
|     $50-100|          6306|
|       >$200|          4765|
|        <$50|          1693|
+------------+--------------+



In [10]:
# Reviews vs Price Analysis
spark.sql("""
SELECT 
    number_of_reviews, 
    ROUND(AVG(price_per_night), 2) as avg_price
FROM airbnb
GROUP BY number_of_reviews
ORDER BY number_of_reviews DESC
LIMIT 20
""").show()

print("✅ EDA Complete — Data ready for visualization phase.")

+-----------------+---------+
|number_of_reviews|avg_price|
+-----------------+---------+
|             1865|    144.0|
|             1618|    163.0|
|             1574|    148.0|
|             1201|    177.0|
|             1188|    161.0|
|             1139|    106.0|
|             1128|    134.0|
|             1048|     88.0|
|              991|    185.0|
|              787|     71.0|
|              761|    185.0|
|              745|    255.0|
|              717|    145.0|
|              707|     95.0|
|              705|     96.0|
|              697|     86.0|
|              679|     44.0|
|              664|     72.0|
|              660|     43.0|
|              647|     63.0|
+-----------------+---------+

✅ EDA Complete — Data ready for visualization phase.
